### Data Loading & Cleaning

In [ ]:
import pandas as pd

df = pd.read_csv("datasetSentences.txt", sep='\t')
df.head()

,sentence_index,sentence
0,1,The Rock is destined to be the 21st Century 's...
1,2,The gorgeously elaborate continuation of `` Th...
2,3,Effective but too-tepid biopic
3,4,If you sometimes like to go to the movies to h...
4,5,"Emerges as something rare , an issue movie tha..."


In [ ]:
from collections import OrderedDict
sentence_idx = OrderedDict()
with open('dictionary.txt', 'r') as f:
  for line in f:
    splits = line.split("|")
    if splits[0] in list(df['sentence']):
      sentence_idx.update({splits[0] : [int(splits[1].strip())]})


In [ ]:
sentiment_values = OrderedDict()
with open('sentiment_labels.txt', 'r') as f:
  for line in f:
    splits = line.split("|")
    try:
      sentiment_values.update({int(splits[0]) : float(splits[1].strip())})
    except:
      continue


In [ ]:
for i in range(len(df)):
  if df.loc[i]['sentence'] in sentence_idx:
    sentence_idx[df.loc[i]['sentence']].append(int(sentiment_values[int(sentence_idx[df.loc[i]['sentence']][0])] * 10 // 2.000000000000001))

In [ ]:
df_mod = pd.DataFrame({'Sentences' : list(sentence_idx.keys()), 'Labels' : [i[1] for i in list(sentence_idx.values())]})

In [ ]:
df_mod['Labels'].value_counts()

1    2967
3    2962
2    2141
4    1772
0    1430
Name: Labels, dtype: int64

In [ ]:
df_mod.shape

(11272, 2)

### Dataset Preparation

In [ ]:
import random
import torch, torchtext
from torchtext import data

In [ ]:
# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [ ]:
Sentence = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [ ]:
fields = [('sentence', Sentence), ('label', Label)]

In [ ]:
example = [torchtext.legacy.data.Example.fromlist([df_mod.Sentences[i],df_mod.Labels[i]], fields) for i in range(df_mod.shape[0])] 

In [ ]:
SSTDataset = torchtext.legacy.data.Dataset(example, fields)

In [ ]:
(train, valid) = SSTDataset.split(split_ratio=[70, 30], random_state = random.seed(SEED))

In [ ]:
Sentence.build_vocab(train)
Label.build_vocab(train)

In [ ]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  16431
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 7445), (',', 6609), ('the', 5571), ('of', 4067), ('and', 4065), ('a', 4035), ('to', 2757), ('-', 2472), ("'s", 2349), ('is', 2315)]
Labels :  defaultdict(None, {1: 0, 3: 1, 2: 2, 4: 3, 0: 4})


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.sentence),
                                                            sort_within_batch=True, device = device)

In [ ]:
next(iter(train_iterator))


[torchtext.legacy.data.batch.Batch of size 32]
	[.sentence]:('[torch.LongTensor of size 32x29]', '[torch.LongTensor of size 32]')
	[.label]:[torch.LongTensor of size 32]

In [ ]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Sentence.vocab.stoi, tokens)

###Model Preparation

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.log_softmax(dense_outputs[0], dim=1)
            
        return output

In [ ]:
# Define hyperparameters
size_of_vocab = len(Sentence.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [ ]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(16431, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 5,171,405 trainable parameters


In [ ]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.NLLLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

### Training Pipeline

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        sentence, sentence_lengths = batch.sentence  
        
        # convert to 1D tensor
        predictions = model(sentence, sentence_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            sentence, sentence_lengths = batch.sentence
            
            # convert to 1d tensor
            predictions = model(sentence, sentence_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.572 | Train Acc: 27.55%
	 Val. Loss: 1.574 |  Val. Acc: 26.62% 

	Train Loss: 1.540 | Train Acc: 31.76%
	 Val. Loss: 1.565 |  Val. Acc: 27.84% 

	Train Loss: 1.479 | Train Acc: 36.88%
	 Val. Loss: 1.534 |  Val. Acc: 31.51% 

	Train Loss: 1.351 | Train Acc: 43.87%
	 Val. Loss: 1.484 |  Val. Acc: 34.76% 

	Train Loss: 1.184 | Train Acc: 52.09%
	 Val. Loss: 1.503 |  Val. Acc: 33.89% 

	Train Loss: 1.019 | Train Acc: 60.03%
	 Val. Loss: 1.547 |  Val. Acc: 35.29% 

	Train Loss: 0.862 | Train Acc: 67.35%
	 Val. Loss: 1.621 |  Val. Acc: 34.61% 

	Train Loss: 0.724 | Train Acc: 73.91%
	 Val. Loss: 1.761 |  Val. Acc: 34.20% 

	Train Loss: 0.590 | Train Acc: 79.36%
	 Val. Loss: 1.856 |  Val. Acc: 34.61% 

	Train Loss: 0.474 | Train Acc: 85.05%
	 Val. Loss: 1.974 |  Val. Acc: 34.87% 



### Inference

In [ ]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_sentence(sentence):
    
    categories = {0: "Very Negative", 1:"Negative", 2:"Neutral", 3:"Positive", 4:"Very Positive"}
    
    # tokenize the sentence 
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
classify_sentence("A valid explanation for why Trump won't let women on the golf course.")

'Very Negative'